# mzcn

中文版本的matchzoo-py

本库包是基于matchzoo-py的库包做的二次开发开源项目，MatchZoo 是一个通用的文本匹配工具包，它旨在方便大家快速的实现、比较、以及分享最新的深度文本匹配模型。
<br>
由于matchzoo-py面向英文预处理较为容易，中文处理则需要进行一定的预处理。为此本人在借鉴学习他人成功的基础上，改进了matchzoo-py包，开发mzcn库包。
<br>
mzcn库包对中文文本语料进行只保留文本、去除表情、去除空格、去除停用词等操作，使得使用者可以快速进行中文文本语料进行预处理，使用方法和matchzoo-py基本一致。

# 快速入手

## 定义损失函数和指标

In [1]:
import torch
import numpy as np
import pandas as pd
import mzcn as mz
print('matchzoo version', mz.__version__)
ranking_task = mz.tasks.Ranking(losses=mz.losses.RankHingeLoss())
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]
print("`ranking_task` initialized with metrics", ranking_task.metrics)

C:\Users\Administrator\Anaconda3\lib\requests\__init__.py:80: RequestsDependencyWarning: urllib3 (1.25.11) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


matchzoo version 1.0.2
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]


## 准备输入数据

In [2]:
def load_data(tmp_data,tmp_task):
	df_data = mz.pack(tmp_data,task=tmp_task)
	return df_data
##数据集，并且进行相应的预处理
train=pd.read_csv('./data/train_data.csv').sample(100)
dev=pd.read_csv('./data/dev_data.csv').sample(50)
test=pd.read_csv('./data/test_data.csv').sample(30)
train_pack_raw = load_data(train,ranking_task)
dev_pack_raw = load_data(dev,ranking_task)
test_pack_raw=load_data(test,ranking_task)

# 停用表配置

In [3]:
import os

In [5]:
folder=mz.__path__[0]+'\\preprocessors\\units\\'
file=folder+'stopwords.txt'
if not os.path.exists(file):
    print('请将stopwords.txt放置在'+folder+'下面')
else:
    print('停用表配置成功')

停用表配置成功


## 数据集预处理

In [6]:
preprocessor = mz.models.aNMM.get_default_preprocessor()

In [7]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of ChineseRemoveBlack => ChineseSimplified => ChineseEmotion => IsChinese => ChineseStopRemoval => ChineseTokenizeDemo => Tokenize => Lowercase => PuncRemoval:   0%| | 0/93 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.871 seconds.
Prefix dict has been built succesfully.
Processing text_left with chain_transform of ChineseRemoveBlack => ChineseSimplified => ChineseEmotion => IsChinese => ChineseStopRemoval => ChineseTokenizeDemo => Tokenize => Lowercase => PuncRemoval: 100%|█| 93/93 [00:02<00:00, 37.69it/s]
Processing text_right with chain_transform of ChineseRemoveBlack => ChineseSimplified => ChineseEmotion => IsChinese => ChineseStopRemoval => ChineseTokenizeDemo => Tokenize => Lowercase => PuncRemoval: 100%|█| 96/96 [00:00<00:00, 174.01it/s]
Building Vocabulary from a datapack.: 100%|██████████████████████████████████████| 9

## 生成训练数据

In [8]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1
)
devset = mz.dataloader.Dataset(
    data_pack=dev_pack_processed
)

## 生成管道

In [9]:
padding_callback = mz.models.aNMM.get_default_padding_callback()

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    stage='train',
    callback=padding_callback,
)
devloader = mz.dataloader.DataLoader(
    dataset=devset,
    stage='dev',
    callback=padding_callback,
)

## 定义模型

In [10]:
model = mz.models.aNMM()
model.params['task'] = ranking_task
model.params["embedding_output_dim"]=100
model.params["embedding_input_dim"]=preprocessor.context["embedding_input_dim"]
model.params['dropout_rate'] = 0.1
model.build()
print(model)

aNMM(
  (embedding): Embedding(330, 100, padding_idx=0)
  (matching): Matching()
  (hidden_layers): Sequential(
    (0): Sequential(
      (0): Linear(in_features=200, out_features=100, bias=True)
      (1): ReLU()
    )
    (1): Sequential(
      (0): Linear(in_features=100, out_features=1, bias=True)
      (1): ReLU()
    )
  )
  (q_attention): Attention(
    (linear): Linear(in_features=100, out_features=1, bias=False)
  )
  (dropout): Dropout(p=0.1, inplace=False)
  (out): Linear(in_features=1, out_features=1, bias=True)
)


## 模型训练

In [11]:
optimizer = torch.optim.Adam(model.parameters(), lr = 3e-4)

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=devloader,
    validate_interval=None,
    epochs=10
)

trainer.run()

[Iter-1 Loss-1.000]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2188 - normalized_discounted_cumulative_gain@5(0.0): 0.2188 - mean_average_precision(0.0): 0.2188



[Iter-2 Loss-1.000]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2188 - normalized_discounted_cumulative_gain@5(0.0): 0.2188 - mean_average_precision(0.0): 0.2188



[Iter-3 Loss-1.000]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2188 - normalized_discounted_cumulative_gain@5(0.0): 0.2188 - mean_average_precision(0.0): 0.2188



[Iter-4 Loss-1.000]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2188 - normalized_discounted_cumulative_gain@5(0.0): 0.2188 - mean_average_precision(0.0): 0.2188



[Iter-5 Loss-1.000]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2188 - normalized_discounted_cumulative_gain@5(0.0): 0.2188 - mean_average_precision(0.0): 0.2188



[Iter-6 Loss-1.000]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2188 - normalized_discounted_cumulative_gain@5(0.0): 0.2188 - mean_average_precision(0.0): 0.2188



[Iter-7 Loss-1.000]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2188 - normalized_discounted_cumulative_gain@5(0.0): 0.2188 - mean_average_precision(0.0): 0.2188



[Iter-8 Loss-1.000]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2188 - normalized_discounted_cumulative_gain@5(0.0): 0.2188 - mean_average_precision(0.0): 0.2188



[Iter-9 Loss-1.000]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2188 - normalized_discounted_cumulative_gain@5(0.0): 0.2188 - mean_average_precision(0.0): 0.2188



[Iter-10 Loss-1.000]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.2188 - normalized_discounted_cumulative_gain@5(0.0): 0.2188 - mean_average_precision(0.0): 0.2188

Cost time: 3.2511348724365234s


# Install

由于mzcn是依赖于matchzoo-py模型，所以一共有两种途径安装mzcn

### Install MatchZoo-py from Pypi:
pip install mzcn

### Install MatchZoo-py from the Github source:

git clone https://github.com/yingdajun/mzcn.git
<br>
cd mzcn
<br>
python setup.py install

# Citation

本人是第一次写库包，水平有限，希望能给大家带来使用的帮助，如果有不足的地方请多指教
这里是所有引用过的库包

## matchzoo-py

@inproceedings{Guo:2019:MLP:3331184.3331403,
 author = {Guo, Jiafeng and Fan, Yixing and Ji, Xiang and Cheng, Xueqi},
 title = {MatchZoo: A Learning, Practicing, and Developing System for Neural Text Matching},
 booktitle = {Proceedings of the 42Nd International ACM SIGIR Conference on Research and Development in Information Retrieval},
 series = {SIGIR'19},
 year = {2019},
 isbn = {978-1-4503-6172-9},
 location = {Paris, France},
 pages = {1297--1300},
 numpages = {4},
 url = {http://doi.acm.org/10.1145/3331184.3331403},
 doi = {10.1145/3331184.3331403},
 acmid = {3331403},
 publisher = {ACM},
 address = {New York, NY, USA},
 keywords = {matchzoo, neural network, text matching},
} 

## CSDN的作者：SK-Berry的博文

https://blog.csdn.net/sk_berry/article/details/104984599